In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
import joblib

def train_model():
    print("Iniciando Protocolo de Entrenamiento - Proyecto Wow...")
    
    # 1. Cargar Datos
    try:
        df = pd.read_csv("nhanes_labeled_data.csv")
    except FileNotFoundError:
        print("Error: Falta 'nhanes_labeled_data.csv'")
        return

    # 2. Preprocesamiento
    # Separamos Features (X) y Target (y)
    # Eliminamos 'ID' porque es un identificador, no un predictor (evitar overfitting espurio)
    X = df.drop(columns=['Diagnosis', 'ID']) 
    y_raw = df['Diagnosis']
    
    # Codificar Target (Texto -> Números)
    # XGBoost necesita 0, 1, 2, 3...
    le = LabelEncoder()
    y = le.fit_transform(y_raw)
    
    # Guardamos el encoder para que la App sepa qué es 0, 1, 2...
    joblib.dump(le, 'label_encoder.joblib')
    print(f"Clases detectadas: {le.classes_}")

    # 3. Configuración del Modelo (XGBoost Classifier)
    # Usamos 'scale_pos_weight' implícito o métricas logloss para multiclase
    model = xgb.XGBClassifier(
        objective='multi:softprob', # Probabilidad para cada clase
        num_class=len(le.classes_), # 4 clases
        eval_metric='mlogloss',
        use_label_encoder=False,
        random_state=42,
        n_estimators=100,           # Árboles
        max_depth=4,                # Profundidad (evitar overfitting)
        learning_rate=0.1
    )

    # 4. Validación Cruzada Estratificada (Mandamiento #4)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    f1_scores = []
    recalls = []
    
    print("\nComenzando Stratified K-Fold (5 Folds)...")
    print("-" * 60)
    
    fold = 1
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Entrenar
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_test)
        
        # Métricas del Fold
        # Usamos 'weighted' para dar peso según la cantidad de ejemplos reales
        f1 = f1_score(y_test, y_pred, average='weighted')
        acc = accuracy_score(y_test, y_pred)
        
        f1_scores.append(f1)
        
        print(f"🔹 Fold {fold}: Accuracy={acc:.4f} | F1-Weighted={f1:.4f}")
        fold += 1

    print("-" * 60)
    print(f"Promedio F1-Weighted: {np.mean(f1_scores):.4f} (+/- {np.std(f1_scores):.4f})")
    
    # 5. Entrenamiento Final (Full Data) y Reporte Detallado
    print("\nEntrenando modelo final con TODOS los datos...")
    model.fit(X, y)
    
    # Guardamos el modelo
    model.save_model("xgboost_clinical_v1.json")
    print("💾 Modelo guardado como 'xgboost_clinical_v1.json'")
    
    # Reporte de Confusión (Sobre todo el dataset para ver debilidades)
    # OJO: Esto es 'evaluación en train' (solo informativo, ya validamos con CV)
    final_preds = model.predict(X)
    print("\n🔍 Reporte de Clasificación (Training Set - Sanity Check):")
    print(classification_report(y, final_preds, target_names=le.classes_))
    
    print("Pipeline finalizado. Listo para Fase 4 (Explicabilidad).")

if __name__ == "__main__":
    train_model()

Iniciando Protocolo de Entrenamiento - Proyecto Wow...
Clases detectadas: ['Anemia' 'Infection' 'Normal' 'Thrombocytopenia']

Comenzando Stratified K-Fold (5 Folds)...
------------------------------------------------------------


c:\Users\matir\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:02:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 Fold 1: Accuracy=0.9987 | F1-Weighted=0.9987


c:\Users\matir\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:02:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 Fold 2: Accuracy=0.9987 | F1-Weighted=0.9987


...

🔹 Fold 4: Accuracy=0.9980 | F1-Weighted=0.9980


c:\Users\matir\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:02:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 Fold 5: Accuracy=0.9967 | F1-Weighted=0.9967
------------------------------------------------------------
Promedio F1-Weighted: 0.9980 (+/- 0.0007)

Entrenando modelo final con TODOS los datos...


c:\Users\matir\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:02:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


💾 Modelo guardado como 'xgboost_clinical_v1.json'

🔍 Reporte de Clasificación (Training Set - Sanity Check):
                  precision    recall  f1-score   support

          Anemia       1.00      1.00      1.00      1060
       Infection       1.00      1.00      1.00       440
          Normal       1.00      1.00      1.00      5792
Thrombocytopenia       1.00      1.00      1.00       235

        accuracy                           1.00      7527
       macro avg       1.00      1.00      1.00      7527
    weighted avg       1.00      1.00      1.00      7527

Pipeline finalizado. Listo para Fase 4 (Explicabilidad).
